# Домашнее задание 3. Парсинг, Git и тестирование на Python

**Цели задания:**

* Освоить базовые подходы к web-scraping с библиотеками `requests` и `BeautisulSoup`: навигация по страницам, извлечение HTML-элементов, парсинг.
* Научиться автоматизировать задачи с использованием библиотеки `schedule`.
* Попрактиковаться в использовании Git и оформлении проектов на GitHub.
* Написать и запустить простые юнит-тесты с использованием `pytest`.


В этом домашнем задании вы разработаете систему для автоматического сбора данных о книгах с сайта [Books to Scrape](http://books.toscrape.com). Нужно реализовать функции для парсинга всех страниц сайта, извлечения информации о книгах, автоматического ежедневного запуска задачи и сохранения результата.

Важной частью задания станет оформление проекта: вы создадите репозиторий на GitHub, оформите `README.md`, добавите артефакты (код, данные, отчеты) и напишете базовые тесты на `pytest`.



In [2]:
! pip install -q schedule pytest

In [3]:
# Библиотеки, которые могут вам понадобиться
# При необходимости расширяйте список
import time
import requests
import schedule
from bs4 import BeautifulSoup
import re
import json
import os

## Задание 1. Сбор данных об одной книге (20 баллов)

В этом задании мы начнем подготовку скрипта для парсинга информации о книгах со страниц каталога сайта [Books to Scrape](https://books.toscrape.com/).

Для начала реализуйте функцию `get_book_data`, которая будет получать данные о книге с одной страницы (например, с [этой](http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html)). Соберите всю информацию, включая название, цену, рейтинг, количество в наличии, описание и дополнительные характеристики из таблицы Product Information. Результат достаточно вернуть в виде словаря.

**Не забывайте про соблюдение PEP-8** — помимо качественно написанного кода важно также документировать функции по стандарту:
* кратко описать, что она делает и для чего нужна;
* какие входные аргументы принимает, какого они типа и что означают по смыслу;
* аналогично описать возвращаемые значения.

*P. S. Состав, количество аргументов функции и тип возвращаемого значения можете менять как вам удобно. То, что написано ниже в шаблоне — лишь пример.*

In [4]:
def get_book_data(book_url: str) -> dict:
   """
   Функция осуществляет сбор информации со страницы каталога Books to Scrape
   на вход (Args): валидный URL страницы каталога
   на выходе (Returns): словарь со следующими данными:
   - 'title' (str): название книги.
   - 'price' (str): стоимость книги (в евро, в формате ЕЕ.ЦЦ - евро, центы)
   - 'rating' (int): рейтинг книги (значение от 1 до 5).
   - 'availability' (int): количество книг в наличии
   - 'description' (str): описание книги
   - 'product_info' (dict): доп. характеристики
   исключения (Raises):
   - requests.RequestException - Не удалось загрузить страницу
   - AttributeError - Структура страницы не соответствует ожидаемой
   """

   # НАЧАЛО ВАШЕГО РЕШЕНИЯ
   # Загрузка html-страницы
   response = requests.get(book_url)
   response.raise_for_status() # Вызов исключения в случае http-ошибки
   response.encoding = 'utf-8'
   soup = BeautifulSoup(response.text, 'html.parser')

   # Извлечение названия книги
   title = soup.find('h1').text.strip()

   # Извлечение стоимости с помощью регулярного выражения (без указания валюты)
   price_element = soup.find('p', class_='price_color')
   if price_element:
      price_text = price_element.text.strip()
      # Поиск числа в формате ЕЕ.ЦЦ
      match = re.search(r'(\d+\.\d+)', price_text)
      price = match.group(1) if match else None
   else:
      price = None

   # Извлечение рейтинга в виде числа
   rating_classes = {'One': 1, 'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5}
   rating_element = soup.find('p', class_='star-rating')
   rating_text = rating_element['class'][1] if rating_element and len(rating_element['class']) > 1 else None
   rating = rating_classes.get(rating_text, None)

   # Извлечение количества книг в наличии
   availability_element = soup.find('p', class_='instock')
   if availability_element:
      availability_text = availability_element.text.strip()
      # Извлечение числа из строки "In stock (22 available)"
      availability = int(availability_text.split('(')[1].split()[0])
   else:
      availability = None

   # Извлечение описания книги
   description_div = soup.find('div', id='product_description')
   description = description_div.find_next('p').text.strip() if description_div else None

   # Извлечение доп. характеристик
   product_info = {}
   table = soup.find('table', class_='table table-striped')
   if table:
      rows = table.find_all('tr')
      for row in rows:
         key = row.find('th').text.strip()
         value = row.find('td').text.strip()
         # Очищение цен и налогов от символов валюты
         match = re.search(r'(\d+\.\d+)', value)
         if match:
            value = match.group(1)
         product_info[key] = value

   # Формирование результата
   book_data = {
      'title': title,
      'price': price,
      'rating': rating,
      'availability': availability,
      'description': description,
      'product_info': product_info
   }

   return book_data
   # КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [5]:
# Используйте для самопроверки
book_url = 'http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'
get_book_data(book_url)

{'title': 'A Light in the Attic',
 'price': '51.77',
 'rating': 3,
 'availability': 22,
 'description': "It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love th It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love that Silverstein. Need proof of his genius? RockabyeRockabye baby, in the treetopDon't you know a treetopIs no safe place to rock?And who put you 

## Задание 2. Сбор данных обо всех книгах (20 баллов)

Создайте функцию `scrape_books`, которая будет проходиться по всем страницам из каталога (вида `http://books.toscrape.com/catalogue/page-{N}.html`) и осуществлять парсинг всех страниц в цикле, используя ранее написанную `get_book_data`.

Добавьте аргумент-флаг, который будет отвечать за сохранение результата в файл: если он будет равен `True`, то информация сохранится в ту же папку в файл `books_data.txt`; иначе шаг сохранения будет пропущен.

**Также не забывайте про соблюдение PEP-8**

In [6]:
def scrape_books(is_save: bool = False) -> list[dict]:
    """
    Функция осуществляет парсинг всего каталога Books to Scrape,
    используя функцию get_book_data
    на вход (Args): флаг необходимости сохранять данные в файл (по умолчанию - False)
    на выходе (Returns): список словарей с данными о книгах
    Исключения (Raises): requests.RequestException - Не удалось загрузить страницу
    """

    # НАЧАЛО ВАШЕГО РЕШЕНИЯ
    url = "http://books.toscrape.com/catalogue/"
    books_data = []
    page_num = 1

    while True:
        # Формирование постраничного url внутри каталога 
        page_url = f"{url}page-{page_num}.html"
        response = requests.get(page_url)

        # Если страница не найдена, прерываем цикл
        if response.status_code == 404:
            break

        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        # Поиск всех ссылок по тегам
        book_links = soup.find_all('h3')
        for h3 in book_links:
            link = h3.find('a')
            if link:
                book_url = url + link['href']
                # Парсинг данных книги
                book_data = get_book_data(book_url)
                books_data.append(book_data)

        page_num += 1

    # Сохранение в файл
    if is_save:
        file_path = os.path.join(os.getcwd(), 'books_data.txt')
        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(books_data, f, indent=4, ensure_ascii=False)

    return books_data
    # КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [7]:
# Проверка работоспособности функции
res = scrape_books(is_save=True) # Допишите ваши аргументы
print(type(res), len(res)) # и проверки

print("Первые 3 книги:")
for i, book in enumerate(res[:3]):
    print(f"{i+1}. Title: {book.get('title', 'N/A')}, Price: {book.get('price', 'N/A')}")

if os.path.exists('books_data.txt'):
    print("Файл 'books_data.txt' создан успешно.")
    with open('books_data.txt', 'r', encoding='utf-8') as f:
        lines = f.readlines()[3:10]
        for line in lines:
            print(line.strip())
else:
    print("Файл не найден.")

<class 'list'> 1000
Первые 3 книги:
1. Title: A Light in the Attic, Price: 51.77
2. Title: Tipping the Velvet, Price: 53.74
3. Title: Soumission, Price: 50.10
Файл 'books_data.txt' создан успешно.
"price": "51.77",
"rating": 3,
"availability": 22,
"description": "It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love th It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and sm

## Задание 3. Настройка регулярной выгрузки (10 баллов)

Настройте автоматический запуск функции сбора данных каждый день в 19:00.
Для автоматизации используйте библиотеку `schedule`. Функция должна запускаться в указанное время и сохранять обновленные данные в текстовый файл.



Бесконечный цикл должен обеспечивать постоянное ожидание времени для запуска задачи и выполнять ее по расписанию. Однако чтобы не перегружать систему, стоит подумать о том, чтобы выполнять проверку нужного времени не постоянно, а раз в какой-то промежуток. В этом вам может помочь `time.sleep(...)`.

Проверьте работоспособность кода локально на любом времени чч:мм.



In [8]:
# НАЧАЛО ВАШЕГО РЕШЕНИЯ
# Настройка расписания на каждый день
schedule.every().day.at("19:00").do(scrape_books, is_save=True)

# Проверка необходимости запуска каждые 60 сек
while True:
    schedule.run_pending()
    time.sleep(60)
# КОНЕЦ ВАШЕГО РЕШЕНИЯ

## Задание 4. Написание автотестов (15 баллов)

Создайте минимум три автотеста для ключевых функций парсинга — например, `get_book_data` и `scrape_books`. Идеи проверок (можете использовать свои):

* данные о книге возвращаются в виде словаря с нужными ключами;
* список ссылок или количество собранных книг соответствует ожиданиям;
* значения отдельных полей (например, `title`) корректны.

Оформите тесты в отдельном скрипте `tests/test_scraper.py`, используйте библиотеку `pytest`. Убедитесь, что тесты проходят успешно при запуске из терминала командой `pytest`.

Также выведите результат их выполнения в ячейке ниже.

**Не забывайте про соблюдение PEP-8**


In [39]:
# Ячейка для демонстрации работоспособности
# Сам код напишите в отдельном скрипте
! pytest test/test_scraper.py

============================= test session starts =============================
platform win32 -- Python 3.13.5, pytest-8.3.4, pluggy-1.5.0
rootdir: c:\Users\ASHiryaeva\Downloads
plugins: anyio-4.7.0
collected 0 items / 1 error

=================================== ERRORS ====================================
____________________ ERROR collecting test/test_scraper.py ____________________
ImportError while importing test module 'c:\Users\ASHiryaeva\Downloads\test\test_scraper.py'.
Hint: make sure your test modules/packages have valid Python names.
Traceback:
..\AppData\Local\anaconda3\Lib\importlib\__init__.py:88: in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
test\test_scraper.py:2: in <module>
    from scraper import get_book_data, scrape_books
E   ModuleNotFoundError: No module named 'scraper'
=========================== short test summary info ===========================
ERROR test/test_scraper.py
!!!!!!!!!!!!!!!!!!! Interrupted: 1 error during collec

## Задание 5. Оформление проекта на GitHub и работа с Git (35 баллов)

В этом задании нужно воспользоваться системой контроля версий Git и платформой GitHub для хранения и управления своим проектом. **Ссылку на свой репозиторий пришлите в форме для сдачи ответа.**

### Пошаговая инструкция и задания

**1. Установите Git на свой компьютер.**

* Для Windows: [скачайте установщик](https://git-scm.com/downloads) и выполните установку.
* Для macOS:

  ```
  brew install git
  ```
* Для Linux:

  ```
  sudo apt update
  sudo apt install git
  ```

**2. Настройте имя пользователя и email.**

Это нужно для подписи ваших коммитов, сделайте в терминале через `git config ...`.

**3. Создайте аккаунт на GitHub**, если у вас его еще нет:
[https://github.com](https://github.com)

**4. Создайте новый репозиторий на GitHub:**

* Найдите кнопку **New repository**.
* Укажите название, краткое описание, выберите тип **Public** (чтобы мы могли проверить ДЗ).
* Не ставьте галочку Initialize this repository with a README.

**5. Создайте локальную папку с проектом.** Можно в терминале, можно через UI, это не имеет значения.

**6. Инициализируйте Git в этой папке.** Здесь уже придется воспользоваться некоторой командой в терминале.

**7. Привяжите локальный репозиторий к удаленному на GitHub.**

**8. Создайте ветку разработки.** По умолчанию вы будете находиться в ветке `main`, создайте и переключитесь на ветку `hw-books-parser`.

**9. Добавьте в проект следующие файлы и папки:**

* `scraper.py` — ваш основной скрипт для сбора данных.
* `README.md` — файл с кратким описанием проекта:

  * цель;
  * инструкции по запуску;
  * список используемых библиотек.
* `requirements.txt` — файл со списком зависимостей, необходимых для проекта (не присылайте все из глобального окружения, создайте изолированную виртуальную среду, добавьте в нее все нужное для проекта и получите список библиотек через `pip freeze`).
* `artifacts/` — папка с результатами парсинга (`books_data.txt` — полностью или его часть, если весь не поместится на GitHub).
* `notebooks/` — папка с заполненным ноутбуком `HW_03_python_ds_2025.ipynb` и запущенными ячейками с выводами на экран.
* `tests/` — папка с тестами на `pytest`, оформите их в формате скрипта(-ов) с расширением `.py`.
* `.gitignore` — стандартный файл, который позволит исключить временные файлы при добавлении в отслеживаемые (например, `__pycache__/`, `.DS_Store`, `*.pyc`, `venv/` и др.).


**10. Сделайте коммит.**

**11. Отправьте свою ветку на GitHub.**

**12. Создайте Pull Request:**

* Перейдите в репозиторий на GitHub.
* Нажмите кнопку **Compare & pull request**.
* Укажите, что было добавлено, и нажмите **Create pull request**.

**13. Выполните слияние Pull Request:**

* Убедитесь, что нет конфликтов.
* Нажмите **Merge pull request**, затем **Confirm merge**.

**14. Скачайте изменения из основной ветки локально.**



### Требования к итоговому репозиторию

* Файл `scraper.py` с рабочим кодом парсера.
* `README.md` с описанием проекта и инструкцией по запуску.
* Папка `artifacts/` с результатом сбора данных (`.txt` файл).
* Папка `tests/` с тестами на `pytest`.
* Папка `notebooks/` с заполненным ноутбуком `HW_03_python_ds_2025.ipynb`.
* Pull Request с комментарием из ветки `hw-books-parser` в ветку `main`.
* Примерная структура:

  ```
  books_scraper/
  ├── artifacts/
  │   └── books_data.txt
  ├── notebooks/
  │   └── HW_03_python_ds_2025.ipynb
  ├── scraper.py
  ├── README.md
  ├── tests/
  │   └── test_scraper.py
  ├── .gitignore
  └── requirements.txt
  ```